In [4]:
# sample reference genomes
import os
import random
all_targets = os.listdir('/home/hermuba/data0118/refseq/prokaryotes/faa')
all_targets = [f for f in all_targets if f.endswith('faa')]
selected = random.sample(all_targets, 50)

In [6]:
import shutil
indir='/home/hermuba/data0118/refseq/prokaryotes/faa'
outdir='/home/hermuba/co-inheritance/test_data/target_genomes'
for s in selected:
    shutil.copyfile(os.path.join(indir, s), os.path.join(outdir, s))

In [7]:
# sample genes and get its faa file
import pandas as pd
gold_anno = pd.read_pickle('/home/hermuba/data0118/goldstandard/ec_rmplasmid_node_anno_df')

In [11]:
# select only genes with GO term
subset_with_anno = gold_anno.loc[(gold_anno['GO'].notnull()) & (gold_anno['cog_ID'].notnull())]

In [12]:
subset_with_anno.head()

,cog_ID,cog_category,cluster,count,core,loose_best_ARO,loose_ARO,strict_best_ARO,strict_ARO,is_card,resfam,aclame_title,is_aclame,nr,hypo_nr,drug_target,is_drug_target,GO,pathway,domain
562.10576.con.0002_214|562.10576,COG1309,K,Cluster 14599,1579.0,True,NaN,NaN,NaN,NaN,False,NaN,NaN,False,EHO00211.1 HTH-type transcriptional repressor ...,NaN,NaN,False,{GO:0003677},NaN,"{IPR036271, IPR011075, IPR009057, IPR001647}"
562.10576.con.0002_294|562.10576,COG4977,K,Cluster 8059,27.0,False,NaN,NaN,NaN,NaN,False,"{marA, ramA}",protein:plasmid:29688 Length: 328 # NCBI annot...,True,EHN91178.1 hypothetical protein ESNG_03889 [Es...,True,NaN,False,"{GO:0003677, GO:0006355, GO:0003700, GO:0043565}",{Reactome: R-HSA-3899300},"{IPR029062, IPR018060, IPR009057, IPR002818}"
562.10576.con.0004_152|562.10576,COG0385,R,Cluster 4577,1.0,False,NaN,NaN,NaN,NaN,False,NaN,protein:plasmid:128244 Length: 277 # NCBI anno...,True,WP_061092939.1 bile acid:sodium symporter fami...,NaN,NaN,False,{GO:0016020},NaN,"{IPR002657, IPR038770}"
562.10576.con.0004_153|562.10576,COG3246,S,Cluster 11302,1.0,False,NaN,NaN,NaN,NaN,False,NaN,protein:plasmid:17531 Length: 296 # NCBI annot...,True,WP_061092938.1 3-keto-5-aminohexanoate cleavag...,NaN,NaN,False,"{GO:0003824, GO:0016740, GO:0019475}",{KEGG: 00310+2.3.1.247},"{IPR008567, IPR013785}"
562.10576.con.0004_155|562.10576,COG1349,K,Cluster 11406,1.0,False,NaN,NaN,NaN,NaN,False,NaN,protein:plasmid:16029 Length: 253 # NCBI annot...,True,WP_061092936.1 DeoR/GlpR transcriptional regul...,NaN,NaN,False,"{GO:0003700, GO:0006355}",NaN,"{IPR036390, IPR014036, IPR018356, IPR036388, I..."


In [14]:
# get those cogs that are small enough that it's easy to benchmark
cog_counts = subset_with_anno['cog_ID'].value_counts()

In [18]:
cog_counts[(cog_counts<60) & (cog_counts>50)]

COG2814    52
Name: cog_ID, dtype: int64

In [19]:
import networkx as nx
def read_edge(net_name):
    path = '/home/hermuba/data0118/network1122/'
    filename = path + net_name + '_lls_rm_plasmid_edgelist'
    
    # skip first line header!!
    with open(filename, 'rb') as f:
      
        net = nx.read_edgelist(f, nodetype = str, comments = '#')
    return(net)


refseq = read_edge('refseq')


In [22]:
small_subgraphs = [c for c in sorted(nx.connected_components(refseq), key=len, reverse=True) if len(c) == 10]

In [26]:
with_anno = []
for subgraph_nodes in small_subgraphs:
    if gold_anno.loc[list(subgraph_nodes), 'GO'].count() > 7:
        with_anno.extend(list(subgraph_nodes))

In [29]:
#selected 50 genes
len(with_anno)

50

In [32]:
# select faa
from Bio import SeqIO
selected_records = []
with open('/home/hermuba/data0118/cdhit/Escherichia0.70rm_plasmid', 'r') as f:
    for record in SeqIO.parse(f, 'fasta'):
        if record.id in with_anno:
            selected_records.append(record)
        
        

In [35]:
SeqIO.write(selected_records, '../test_data/seqs.faa', 'fasta')
   

50

In [37]:
# also export annotation
gold_anno.loc[with_anno, ['nr', 'cog_ID', 'cog_category', 'GO', 'pathway']].to_csv('../test_data/gene_anno.csv')